In [220]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px 
import scipy
import numpy as np
import plotly.io as pio

In [312]:
car_price = 69000
start_cap = 140000
maintenance_monthly = 500*12*10

In [313]:
downpayment_list = [i for i in range(int(car_price*0.3),car_price,2000)]
final_cost_1 = [(0.0298*1*(car_price - downpayment_list[i])+(car_price - downpayment_list[i])) for i in range(0,len(downpayment_list))]

In [314]:
car_loan_df = pd.DataFrame({"Downpayment" : downpayment_list
                            })

In [315]:
for z in range(1,8):
    car_loan_df["{} Year".format(z)] = [((0.0298*z*(car_price - downpayment_list[i])+(car_price - downpayment_list[i])))+downpayment_list[i]+maintenance_monthly for i in range(0,len(downpayment_list))]

In [316]:
car_melt = pd.melt(car_loan_df, id_vars="Downpayment")

In [317]:
fig = px.line(car_melt,x="Downpayment",y="value",color="variable",labels={
                     "Downpayment": "Initial Downpayment",
                     "value": "Total Cost of Car",
                     "variable": "Loan Term"
                 },title="True Cost of Car with Interest and Monthly Payment")
fig.show()

In [318]:
def fv(monthly, interest_rate, time):
    return (monthly * ((((1 + interest_rate)**time)-1)/(interest_rate)))

def ifv(initial, monthly, interest_rate, time):
    return ((initial*((1+interest_rate)**time)) + (monthly * ((((1 + interest_rate)**time)-1)/(interest_rate))))

In [319]:
def car_tradeoff_calc(car_price,downpayment,years,bank_int,capital_invest,simir):
    total_cost = ((bank_int*years*(car_price - downpayment)+(car_price - downpayment))) + downpayment + maintenance_monthly
    pay_monthly = (total_cost - downpayment)/(years*12)
    pre_invest_monthly = max(0,((capital_invest-downpayment)/(10*12)-pay_monthly))
    pre_invest_total = fv(pre_invest_monthly, simir, years*12)
    post_invest_monthly = ((capital_invest-((pay_monthly+ pre_invest_monthly)*years*12)-downpayment)/((10-years)*12))
    post_invest_total = ifv(pre_invest_total,post_invest_monthly,simir,((10-years)*12))
    total_returns = pre_invest_total + post_invest_total
    return(post_invest_total)

In [320]:
payoff_df = pd.DataFrame({"Downpayment" : downpayment_list
                            })

In [321]:
if (start_cap < car_loan_df.iloc[0,-1]):
    print("Error: Starting capital not large enough")
else:
    for z in range(1,8):
        payoff_df["{} yr".format(z)] = [car_tradeoff_calc(car_price,downpayment_list[i],z,0.0298,start_cap,0.007) for i in range(0,len(downpayment_list))]

In [322]:
payoff_df_melt = pd.melt(payoff_df, id_vars="Downpayment")

In [323]:
fig = px.line(payoff_df_melt,x="Downpayment",y="value",color="variable",labels={
                     "Downpayment": "Initial Downpayment",
                     "value": "Return on Investment",
                     "variable": "Loan Term"
                 }, title="Payoff Diagram, with 10 year capital: {}".format(start_cap))
fig.show()

In [347]:
def car_tradeoff_calc_monthly(car_price,downpayment,years,bank_int,cap_month,simir):
    total_cost = ((bank_int*years*(car_price - downpayment)+(car_price - downpayment))) + downpayment + maintenance_monthly
    pay_monthly = (total_cost - downpayment)/(years*12)
    if cap_month < pay_monthly:
        loss = (pay_monthly - cap_month)*years*12
        post_invest_monthly = cap_month
        post_invest_total = fv(post_invest_monthly, simir, ((10-years)*12))
        total_returns = post_invest_total - loss
        return (total_returns)
    else: 
        pre_invest_monthly = max(0,(cap_month-pay_monthly))
        pre_invest_total = fv(pre_invest_monthly, simir, years*12)
        post_invest_monthly = cap_month
        post_invest_total = ifv(pre_invest_total,post_invest_monthly,simir,((10-years)*12))
        total_returns = pre_invest_total + post_invest_total
        return(post_invest_total)

month_cap = 1200

In [348]:
payoffmonth_df = pd.DataFrame({"Downpayment" : downpayment_list
                            })

In [349]:
for z in range(1,8):
    payoffmonth_df["{} yr".format(z)] = [car_tradeoff_calc_monthly(car_price,downpayment_list[i],z,0.0298,month_cap,0.007) for i in range(0,len(downpayment_list))]

In [350]:
payoffmonth_df_melt = pd.melt(payoffmonth_df, id_vars="Downpayment")

In [353]:
fig = px.line(payoffmonth_df_melt,x="Downpayment",y="value",color="variable",labels={
                     "Downpayment": "Initial Downpayment",
                     "value": "Return on Investment",
                     "variable": "Loan Term"
                 }, title="Payoff Diagram, with monthly budget: {}".format(month_cap))
fig.show()